<a href="https://colab.research.google.com/github/AdityaPrasad275/Detecting-Depression-Through-Tweets/blob/main/LSTM_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Versions used : 

Library  | Version
-------------------|------------------
matplotlib | 3.2.2
pandas  | 1.3.5
seaborn | 0.11.2
nltk | 3.7
keras | 2.9

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
import os
os.chdir('MyDrive/Projects')
!ls

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

#to be able to drop nodes
from keras.layers import Dense, Dropout, Flatten
import re

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import string
file_path='Processed data.csv'
df=pd.read_csv(file_path,on_bad_lines='skip')

In [ ]:
print(df.shape)

(1578550, 6)


In [ ]:
df.head()

In [ ]:
data = df[['Sentiment', 'SentimentText']]
data2=df[['Sentiment','WithoutStopwords']]

In [ ]:
print("Entries with sentiment = 1 is :",data[ data['Sentiment'] == 1].size)
print("Entries with sentiment = 1 is :",data[ data['Sentiment'] == 0].size)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)

In [ ]:
embed_dim = 50
lstm_out = 80
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))

model.add(LSTM(lstm_out, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 50)            100000    
                                                                 
 dropout (Dropout)           (None, 40, 50)            0         
                                                                 
 lstm (LSTM)                 (None, 80)                41920     
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense (Dense)               (None, 2)                 162       
                                                                 
Total params: 142,082
Trainable params: 142,082
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

#31mins to run 1 epoch

33051/33051 - 1852s - loss: 0.4524 - accuracy: 0.7864 - 1852s/epoch - 56ms/step


In [ ]:
validation_size = 150000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

#takes 3mins to run

11545/11545 - 106s - loss: 0.4344 - accuracy: 0.7978 - 106s/epoch - 9ms/step
score: 0.43
acc: 0.80


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    #print(data['text'].iloc[-validation_size+x])
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    #print(result)
    #print(Y_validate[x])
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    #print(np.argmax(Y_validate[x]))
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
    #print(np.argmax(result) == np.argmax(Y_validate[x]))
    #print()
    #print()
    

# print("pos_correct", pos_correct)
# print("neg_correct", neg_correct)


In [ ]:
print("pos_cnt", pos_cnt)
print("neg_cnt", neg_cnt)
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_cnt 75138
neg_cnt 74862
pos_acc 77.79419202001651 %
neg_acc 81.49929203066976 %


In [ ]:
print("pos_cnt", pos_cnt)
print("neg_cnt", neg_cnt)
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_cnt 752
neg_cnt 748
pos_acc 80.45212765957447 %
neg_acc 83.28877005347593 %
